In [8]:
# -*- coding: utf-8 -*-
import bs4 as bs
from urllib import urlopen as uReq
import sys
import string
import pandas as pd

have to create list with the dates I want
    for each date in the list(fechas):
    extract day, month, year from date and create variables
    create the url, with Lanacion+fecha using the 'date' variables
    create the bs object and parse it
    get the data I want from each file
    create a .csv file with Lanacion+fecha
    copy the data into the file
    close the file   

In [9]:
# I request the URL, read, parsed with BS and saved it in the variable soup
date_range = pd.date_range('2017-04-30',periods=3, freq='d')
date_range = date_range.strftime("%d/%m/%Y")

# creates the file, using dd/mm/yyyy in the filename
filename = '/users/hnesman/Downloads/DATA_Science/La Nacion.txt'
    # opens the file
f = open(filename,'w')
    # creates and writes the headers of the file
headers = "Date|Sección|Title|Encabezado\n"
f.write(headers)
secciones = ['c30','c272','c7','c28','c131','c337','c432','c7775']
# sec_dic = {'c30':'política','c272':'economía','c7':'elmundo','c28':'opinión','c131':'deportiva','c337':'elcampo',
#           'c432':'tecnología','c7775':'seguridad'}
for i in date_range:
        for sec in secciones:
        # creates de url link with the date
            url = 'http://servicios.lanacion.com.ar/archivo-f%s-%s' % (i,sec)
            # creates de BS object from the URL
            sauce = uReq(url)
            # does the parsing
            soup = bs.BeautifulSoup(sauce,'html.parser')
            # gets the tabs containing the date from the BS object
            date = soup.findAll('div',{'class':'titFecha'})
            # converts date into string 
            date = str(date)
            # this is to extract the date from the date container
            fecha = '"titFecha">'
            date = date[date.find(fecha)+len(fecha):date.find(fecha)+21]
            # slices date to get dd/mm/yyyy
            day = date[:2]
            month = date[3:5]
            year = date[6:12]
            date = (day+'/'+month+'/'+year)
            
            print date + ' - ' + sec

            # creates the (list?) for the containers of the elements I want
            acumulados = soup.findAll('li',{'class':'acumulados'})
            #creates the loop for each element of the (list?) acumulados
            for acum in acumulados:
                # gets the title (which is the url link)
                title =  acum.a['href']
                print title
                # gets the body of the element
                encabezado = acum.p
                print encabezado,'\n'
                #writes the info (title & encabezado) into the file
                f.write(str(date) + '|'+ str(sec) + '|' + str(title) + '|' + str(encabezado).replace('<p>','').replace('</p>','') + '\n')

f.close()

30/04/2017 - c30
http://www.lanacion.com.ar/2019169-odebrecht-aporto-fondos-para-la-campana-de-pro-y-otros-partidos-estan-bajo-la-lupa
<p>Donó $500.000 con Braskem; otro dirigentes niegan haber recibido aportes</p> 

http://www.lanacion.com.ar/2019170-la-caja-dos-que-financio-la-politica-de-la-region
<p>Odebrecht admitió pagos ilegales para campañas en  Brasil y en otros países</p> 

http://www.lanacion.com.ar/2019186-macri-interviene-en-la-interna-gremial-y-pule-anuncios-laborales
<p>Mañana se mostrará con los sindicatos aliados y anunciaría un proyecto de reconversión de planes sociales y otro de beneficios impositivos a transportistas; la CGT se reúne por separado, igual que la CTA</p> 

http://www.lanacion.com.ar/2019187-la-crisis-de-la-salud-el-otro-conflicto-que-complica-a-vidal
<p>Este año ya hubo 20 días de paro, y muchos hospitales funcionan con varias deficiencias</p> 

http://www.lanacion.com.ar/2019188-la-izquierda-se-une-para-liderar-un-gremio
<p>Quiere destronar al kirchn

In [10]:
LaNacion = pd.read_table('/users/hnesman/Downloads/DATA_Science/La Nacion.txt',delimiter="|")
LaNacion.columns = ['Date', 'SecCode','Title','Encabezado']
sec_dic = {'c30':'política','c272':'economía','c7':'elmundo','c28':'opinión','c131':'deportiva','c337':'elcampo',
          'c432':'tecnología','c7775':'seguridad'}
LaNacion['Sec Label'] = LaNacion.SecCode.apply(lambda x: sec_dic[str(x)])
LaNacion['Media'] = 'La Nacion'
LaNacion = LaNacion.drop('SecCode',axis=1)
LaNacion.columns = ['Date','Title','Encabezado','SecLabel','Media']
LaNacion.to_csv('/users/hnesman/Downloads/DATA_Science/LaNacion.csv')
LaNacion.head()

,Date,Title,Encabezado,SecLabel,Media
0,30/04/2017,http://www.lanacion.com.ar/2019169-odebrecht-a...,Donó $500.000 con Braskem; otro dirigentes nie...,política,La Nacion
1,30/04/2017,http://www.lanacion.com.ar/2019170-la-caja-dos...,Odebrecht admitió pagos ilegales para campañas...,política,La Nacion
2,30/04/2017,http://www.lanacion.com.ar/2019186-macri-inter...,Mañana se mostrará con los sindicatos aliados ...,política,La Nacion
3,30/04/2017,http://www.lanacion.com.ar/2019187-la-crisis-d...,"Este año ya hubo 20 días de paro, y muchos hos...",política,La Nacion
4,30/04/2017,http://www.lanacion.com.ar/2019188-la-izquierd...,Quiere destronar al kirchnerista Iadarola al f...,política,La Nacion


In [5]:
# Date = LaNacion.iloc[:,:1]
# # Date.head()
# Other = LaNacion.iloc[:,2:]
# # Other.head()
# LaNacion = Date.join(Other)
# LaNacion.head()